# Question Answering with Documents using LangChain 🦜️🔗 and Vertex AI Matching Engine

## Generative AI workshop

## 🔗🔗🔗 Connect to the prepared Vector Search / Matching Engine index 🔗🔗🔗

Based on: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb


## Getting Started

Install Vertex AI SDK, other packages and their dependencies
Install the following packages required to execute this notebook.

In [1]:
# Install Vertex AI LLM SDK
! pip install --user --upgrade google-cloud-aiplatform==1.31.0 langchain==0.0.201

# Dependencies required by Unstructured PDF loader
! sudo apt -y -qq install tesseract-ocr libtesseract-dev
! sudo apt-get -y -qq install poppler-utils
! pip install --user unstructured==0.7.5 pdf2image==1.16.3 pytesseract==0.3.10 pdfminer.six==20221105

# For Matching Engine integration dependencies (default embeddings)
! pip install --user tensorflow_hub==0.13.0 tensorflow_text==2.12.1

libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
  Using cached tensorflow_hub-0.13.0-py2.py3-none-any.whl (100 kB)
  Using cached tensorflow_text-2.12.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 37.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warnin

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [3]:
import os
import urllib.request

if not os.path.exists("utils"):
    os.makedirs("utils")

url_prefix = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/use-cases/document-qa/utils"
files = ["__init__.py", "matching_engine.py", "matching_engine_utils.py"]

for fname in files:
    urllib.request.urlretrieve(f"{url_prefix}/{fname}", filename=f"utils/{fname}")

In [4]:
import json
import textwrap
# Utils
import time
import uuid
from typing import List

import numpy as np
import vertexai
# Vertex AI
from google.cloud import aiplatform

print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Langchain
import langchain

print(f"LangChain version: {langchain.__version__}")

from langchain.chains import RetrievalQA
from langchain.document_loaders import GCSDirectoryLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel

# Import custom Matching Engine packages
from utils.matching_engine import MatchingEngine
from utils.matching_engine_utils import MatchingEngineUtils

Vertex AI SDK version: 1.31.0
LangChain version: 0.0.201


In [5]:
PROJECT_ID = "datamass-2023-genai"  # @param {type:"string"}
REGION = "europe-west1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

## Utility functions

In [6]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

## Initialize LangChain Models

In [7]:
# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Embeddings API integrated with langChain
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

## Connect to the pre-built Matching Engine index

In [8]:
ME_REGION = "europe-west1"
ME_INDEX_NAME = "datamass-2023-genai-mwiewior-index"  # @param {type:"string"}
ME_EMBEDDING_DIR = "datamass-2023-genai-mwiewior-bucket"  # @param {type:"string"}
ME_DIMENSIONS = 768  # when using Vertex PaLM Embedding

In [42]:
mengine = MatchingEngineUtils(PROJECT_ID, ME_REGION, ME_INDEX_NAME)

In [49]:
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = mengine.get_index_and_endpoint()

# temp override
ME_INDEX_ENDPOINT_ID = 'projects/716156477321/locations/europe-west1/indexEndpoints/3806790730337222656'

print(f"ME_INDEX_ID={ME_INDEX_ID}")
print(f"ME_INDEX_ENDPOINT_ID={ME_INDEX_ENDPOINT_ID}")

ME_INDEX_ID=projects/716156477321/locations/europe-west1/indexes/7681680812852379648
ME_INDEX_ENDPOINT_ID=projects/716156477321/locations/europe-west1/indexEndpoints/3806790730337222656


In [50]:
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=ME_REGION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID
)

## Validate semantic search with Matching Engine is working

In [51]:
me.similarity_search("how to integrate kedro with AzureML?", k=2)

Waiting


[Document(page_content="Running Kedro on Azure Machine Learning Pipelines\n\nGitHub: https://github.com/getindata/kedro-azureml Documentation: https://kedro-azureml.readthedocs.io/en/stable/\n\nIf your cloud platform of choice is Azure, we've also got your back! Just recently (August 2022) we published the Kedro-AzureML plugin that enables you to run Kedro pipelines in the Azure Machine Learning Pipelines service. Similar to Vertex AI - Azure ML Pipelines is also a fully managed service that provides easy scale-up capabilities. ML Engineers need to set-up compute clusters first, but it’s an easy process, which can be handled without the involvement of the DevOps teams. Once done, Kedro pipelines can be executed on Azure ML Pipelines. Moreover, metrics and trained models can be easily tracked using Azure ML’s built-in MLflow integration, without any additional configuration and our plug-in supports that.", metadata={'source': 'datamass-lab-2023/', 'document_name': 'Running Kedro... ever

In [29]:
me.similarity_search("How to integrate MLflow with Snowflake?", k=2, search_distance=0.4)

Waiting


[Document(page_content='sent/received from the MLflow instance.\n\nSnowflake API integration for setting up a communication channel from the\n\nSnowflake instance to the cloud HTTPS proxy/gateway service where your\n\nMLflow instance is hosted (e.g. Amazon API Gateway, Google Cloud API\n\nGateway or Azure API Management).\n\nSnowflake storage integration to enable your Snowflake instance to upload\n\nartifacts (e.g. serialized models) to the cloud storage (Amazon S3, Azure Blob\n\nStorage, Google Cloud Storage) used by the MLflow instance.\n\nOffline model deployment within Snowflake is handled by the MLflow-Snowflake\n\nplugin, which allows you to deploy models trained in popular frameworks (such as\n\nPyTorch, Scikit-Learn, TensorFlow, LightGBM, XGBoost, ONNX and others)\n\nnatively in Snowflake as User-Defined Functions, which then can be called\n\ndirectly from SQL to efficiently (using vectorization) perform inference and obtain\n\nthe models’ predictions. The plugin is backed by 

## Retrieval based Question/Answering Chain

### Configure Question/Answering Chain with Vector Store using Text

In [30]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 10
SEARCH_DISTANCE_THRESHOLD = 0.6

# Expose index to the retriever
retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
)

In [31]:
template = """SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [32]:
# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [33]:
# Enable for troubleshooting
qa.combine_documents_chain.verbose = True
qa.combine_documents_chain.llm_chain.verbose = True
qa.combine_documents_chain.llm_chain.llm.verbose = True

In [34]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(query, qa=qa, k=NUMBER_OF_RESULTS, search_distance=SEARCH_DISTANCE_THRESHOLD):
    qa.retriever.search_kwargs["search_distance"] = search_distance
    qa.retriever.search_kwargs["k"] = k
    result = qa({"query": query})
    return formatter(result)

### Run QA chain on sample questions

In [35]:
ask("How to integrate kedro with AzureML?")



> Entering new  chain...
Waiting


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: How to integrate kedro with AzureML?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

Running Kedro on Azure Machine Learning Pipelines

GitHub: https://github.com/getindata/kedro-azureml Documentation: https://kedro-azureml.readthedocs.io/en/stable/

If your cloud platform of choice is Azure, we've also got your back! Just recently (August 2022) we published the Kedro-AzureML plugin that enables you to run Kedro pipelines in the Azure Machine Learning Pipelines serv

In [36]:
ask("How to integrate kedro with MLFlow?")



> Entering new  chain...
Waiting


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: How to integrate kedro with MLFlow?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

Z komentarzem [2]: Dodalem info o 0.2.0 release with MLflow support @sylwia.kolpuc@getindata.com

GCP and Azure deployment scenarios are very much the same except for:

API Gateway (GCP) or API Management (Azure) for exposing MLflow API ● MLflow hosting - e.g. App Engine, Cloud Run (GCP) or Azure Container Apps, Azure

Container Instances (Azure)

Online model serving - Vertex Endpoi

In [37]:
ask("How to integrate MLflow with Snowflake?")



> Entering new  chain...
Waiting


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: How to integrate MLflow with Snowflake?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

sent/received from the MLflow instance.

Snowflake API integration for setting up a communication channel from the

Snowflake instance to the cloud HTTPS proxy/gateway service where your

MLflow instance is hosted (e.g. Amazon API Gateway, Google Cloud API

Gateway or Azure API Management).

Snowflake storage integration to enable your Snowflake instance to upload

artifacts (e.g

### Question outside the knowledge domain

In [38]:
ask("What is NFC?")



> Entering new  chain...
Waiting


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: What is NFC?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."



Question: What is NFC?
Helpful Answer:

> Finished chain.

> Finished chain.

> Finished chain.
Query: What is NFC?
................................................................................
................................................................................
Response: I cannot determine the answer to that.
................................................................................

## Reference: LLM without external knowledge base

You can also send questions directly to the PaLM (`text-bison@001`) available in the LangChain.



In [40]:
# Text model instance integrated with langChain
# llm = VertexAI(
#     model_name="text-bison@001",
#     max_output_tokens=1024,
#     temperature=0.2,
#     top_p=0.8,
#     top_k=40,
#     verbose=True,
# )

print(llm("What are some of the pros and cons of Kedro?"))

**Pros:**

* Kedro is a Python-based framework that makes it easy to build and maintain data pipelines.
* It provides a number of features that make it easy to create modular, reusable, and scalable pipelines, such as:
    * A DAG (directed acyclic graph) representation of the pipeline, which makes it easy to visualize and understand the flow of data.
    * A code generator that can automatically generate Python code for the pipeline.
    * A library of reusable components that can be used to build pipelines.
* Kedro is open source and has a large and active community that provides support and resources.

**Cons:**

* Kedro is a relatively new framework, so it may not be as mature as some of the other options available.
* It can be a bit complex to get started with, especially if you are not familiar with Python or data pipelines.
* Kedro does not support all data sources and formats out of the box, so you may need to write custom code to support your specific needs.

Overall, Kedro is

In [41]:
print(llm("How to integrate MLflow with Snowflake?"))

MLflow is an open source platform for managing the end-to-end machine learning lifecycle. It provides a central repository for storing and tracking experiments, as well as tools for deploying models to production. Snowflake is a cloud-based data warehouse that provides a fast and scalable platform for storing and querying data.

To integrate MLflow with Snowflake, you can use the MLflow Snowflake Connector. The connector provides a simple way to connect MLflow to Snowflake and to store and query experiment data in Snowflake.

To use the connector, you first need to install it. You can do this by running the following command:

```
pip install mlflow-snowflake
```

Once the connector is installed, you can connect MLflow to Snowflake by following these steps:

1. Open the MLflow UI.
2. Click on the "Settings" tab.
3. In the "Connections" section, click on the "Add Connection" button.
4. In the "Connection Type" drop-down menu, select "Snowflake".
5. Enter the following information:

* **